# Task 2: Citation-Reference Linking with BERT-base

**Model:** bert-base-uncased (Baseline)

**Task:** Binary classification - Is this the correct bib entry?

---

In [1]:
# Kaggle already has these packages installed!
# No need to install: transformers, datasets, accelerate

# Verify versions (optional)
import transformers, datasets, accelerate
print(f"✅ transformers: {transformers.__version__}")
print(f"✅ datasets: {datasets.__version__}")
print(f"✅ accelerate: {accelerate.__version__}")

✅ transformers: 4.57.1
✅ datasets: 4.4.1
✅ accelerate: 1.11.0


In [2]:
# Data already unzipped by Kaggle - verify it
import os

train_path = '/kaggle/input/thesis-data-task2/train/train'
val_path = '/kaggle/input/thesis-data-task2/val/val'

train_count = len([f for f in os.listdir(train_path) if f.endswith('.in')])
val_count = len([f for f in os.listdir(val_path) if f.endswith('.in')])

print(f"✅ Train: {train_count} files")
print(f"✅ Val: {val_count} files")

✅ Train: 55556 files
✅ Val: 3000 files


In [3]:
# Load data - STREAMING with NEGATIVE SAMPLING (fix class imbalance)
import json
import random
from pathlib import Path
from datasets import IterableDataset

def generate_task2_examples(data_dir, neg_samples=5):
    """
    Generator with negative sampling to fix class imbalance
    
    Args:
        data_dir: Path to data directory
        neg_samples: Number of negative samples per positive (default: 5)
                     This creates 1:5 ratio instead of 1:99
    """
    data_path = Path(data_dir)
    in_files = sorted(data_path.glob("*.in"))
    
    total_files = len(in_files)
    print(f"📊 Found {total_files:,} .in files - streaming mode")
    print(f"⚖️ Using negative sampling: {neg_samples} negatives per positive")

    for i, in_file in enumerate(in_files):
        if (i+1) % 5000 == 0:
            print(f"⏳ Processed {i+1:,}/{total_files:,} files ({(i+1)*100//total_files}%)")
        
        with open(in_file) as f:
            in_data = json.load(f)

        label_file = in_file.with_suffix('.label')
        if not label_file.exists():
            continue
            
        with open(label_file) as f:
            label_data = json.load(f)

        text = in_data.get('text', '')
        bib_entries = in_data.get('bib_entries', {})
        citation_to_bib = label_data.get('correct_citation', {})

        for citation, correct_bib_id in citation_to_bib.items():
            citation_pos = text.find(citation)
            if citation_pos == -1:
                continue

            start = max(0, citation_pos - 200)
            end = min(len(text), citation_pos + len(citation) + 200)
            context = text[start:end]

            # POSITIVE example
            if correct_bib_id in bib_entries:
                bib_data = bib_entries[correct_bib_id]
                bib_text = bib_data.get('abstract', bib_data.get('title', ''))
                yield {
                    'context': context,
                    'bib_entry': bib_text,
                    'label': 1
                }

            # NEGATIVE examples - SAMPLE only N negatives (not all!)
            negative_bib_ids = [bid for bid in bib_entries.keys() if bid != correct_bib_id]
            
            # Sample random negatives (or all if fewer than neg_samples)
            sampled_negatives = random.sample(
                negative_bib_ids, 
                min(neg_samples, len(negative_bib_ids))
            )
            
            for bib_id in sampled_negatives:
                bib_data = bib_entries[bib_id]
                bib_text = bib_data.get('abstract', bib_data.get('title', ''))
                yield {
                    'context': context,
                    'bib_entry': bib_text,
                    'label': 0
                }
    
    print(f"✅ Finished processing all {total_files:,} files")

print("=" * 60)
print("Creating TRAIN dataset (streaming with negative sampling)...")
print("=" * 60)
train_dataset = IterableDataset.from_generator(
    generate_task2_examples, 
    gen_kwargs={'data_dir': '/kaggle/input/thesis-data-task2/train/train', 'neg_samples': 5}
)
print("✅ Train dataset ready")

print("\n" + "=" * 60)
print("Creating VAL dataset (streaming with negative sampling)...")
print("=" * 60)
val_dataset = IterableDataset.from_generator(
    generate_task2_examples,
    gen_kwargs={'data_dir': '/kaggle/input/thesis-data-task2/val/val', 'neg_samples': 5}
)
print("✅ Val dataset ready")

print("\n💡 Using IterableDataset with 1:5 positive:negative ratio!")
print("💡 This prevents class imbalance (instead of 1:99 ratio)")

Creating TRAIN dataset (streaming with negative sampling)...
✅ Train dataset ready

Creating VAL dataset (streaming with negative sampling)...
✅ Val dataset ready

💡 Using IterableDataset with 1:5 positive:negative ratio!
💡 This prevents class imbalance (instead of 1:99 ratio)


In [4]:
# Tokenize - DYNAMIC PADDING (memory efficient)
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    # Use padding=False, let DataCollator handle dynamic padding
    return tokenizer(
        examples['context'],
        examples['bib_entry'],
        max_length=512,
        padding=False,        # Changed from 'max_length' to False
        truncation=True
    )

print("Tokenizing datasets...")
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

print("✅ Tokenization complete (dynamic padding will be applied during training)")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokenizing datasets...
✅ Tokenization complete (dynamic padding will be applied during training)


In [5]:
# Load model
from transformers import AutoModelForSequenceClassification
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

print(f"✅ BERT-base loaded: {model.num_parameters():,} parameters")

2026-01-06 07:34:21.979557: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767684862.176024      23 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767684862.232118      23 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767684862.705552      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767684862.705589      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767684862.705592      23 computation_placer.cc:177] computation placer alr

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ BERT-base loaded: 109,483,778 parameters


In [6]:
# Training setup - OPTIMIZED for IterableDataset
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from pathlib import Path
import os

# Dynamic padding collator (saves VRAM!)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir='/kaggle/working/checkpoints/task2_bert',
    max_steps=10000,              # Use max_steps for IterableDataset (not epochs)
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4, # Effective batch size = 32
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=500,
    eval_strategy='steps',
    eval_steps=500,
    logging_steps=100,
    save_strategy='steps',
    save_steps=500,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    fp16=True,                    # Mixed precision to save VRAM
    report_to='none',
    seed=42
)

# Check for checkpoint to resume from
checkpoint_dir = Path(training_args.output_dir)
checkpoints = sorted(checkpoint_dir.glob('checkpoint-*')) if checkpoint_dir.exists() else []

resume_checkpoint = str(checkpoints[-1]) if checkpoints else None

if resume_checkpoint:
    print(f"🔄 Resuming from: {Path(resume_checkpoint).name}")
else:
    print("🆕 Starting fresh training")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,  # Use dynamic padding!
    compute_metrics=compute_metrics
)

print(f"\n💡 Training config:")
print(f"   - Effective batch size: {8 * 4} (per_device={8} × accumulation={4})")
print(f"   - Max steps: {10000}")
print(f"   - Dynamic padding: ON (saves VRAM)")
print(f"   - FP16: ON (saves VRAM)")

🆕 Starting fresh training

💡 Training config:
   - Effective batch size: 32 (per_device=8 × accumulation=4)
   - Max steps: 10000
   - Dynamic padding: ON (saves VRAM)
   - FP16: ON (saves VRAM)


/tmp/ipykernel_23/39003730.py:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [7]:
# Train
print("="*60)
print("🚀 STARTING TRAINING - BERT-base")
print("="*60)

trainer.train(resume_from_checkpoint=resume_checkpoint)

print("\n✅ Training complete!")

🚀 STARTING TRAINING - BERT-base
📊 Found 55,556 .in files - streaming mode
⚖️ Using negative sampling: 5 negatives per positive


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.397300,0.388601,0.833281,0.528706,0.323413,0.401330
1000,0.382400,0.379331,0.843078,0.682762,0.171509,0.274152
1500,0.384000,0.387789,0.830843,0.512280,0.438364,0.472448
2000,0.369700,0.375551,0.839398,0.553694,0.363611,0.438958
2500,0.387700,0.360685,0.849414,0.628927,0.313399,0.418338
3000,0.371300,0.371965,0.847318,0.600292,0.348237,0.440775
3500,0.398800,0.365411,0.845466,0.811305,0.137659,0.235379
4000,0.359400,0.366501,0.849877,0.703234,0.226939,0.343144
4500,0.361200,0.356791,0.851778,0.685704,0.262482,0.379641
5000,0.381600,0.355087,0.851266,0.649049,0.303103,0.413229


📊 Found 3,000 .in files - streaming mode
⚖️ Using negative sampling: 5 negatives per positive
✅ Finished processing all 3,000 files
📊 Found 3,000 .in files - streaming mode
⚖️ Using negative sampling: 5 negatives per positive
✅ Finished processing all 3,000 files
📊 Found 3,000 .in files - streaming mode
⚖️ Using negative sampling: 5 negatives per positive
✅ Finished processing all 3,000 files
📊 Found 3,000 .in files - streaming mode
⚖️ Using negative sampling: 5 negatives per positive
✅ Finished processing all 3,000 files
⏳ Processed 5,000/55,556 files (8%)
📊 Found 3,000 .in files - streaming mode
⚖️ Using negative sampling: 5 negatives per positive
✅ Finished processing all 3,000 files
📊 Found 3,000 .in files - streaming mode
⚖️ Using negative sampling: 5 negatives per positive
✅ Finished processing all 3,000 files
📊 Found 3,000 .in files - streaming mode
⚖️ Using negative sampling: 5 negatives per positive
✅ Finished processing all 3,000 files
📊 Found 3,000 .in files - streaming mode

In [8]:
# Evaluate
print("📊 VALIDATION RESULTS")
eval_results = trainer.evaluate()
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")

📊 VALIDATION RESULTS
📊 Found 3,000 .in files - streaming mode
⚖️ Using negative sampling: 5 negatives per positive
✅ Finished processing all 3,000 files
eval_loss: 0.3876
eval_accuracy: 0.8315
eval_precision: 0.5144
eval_recall: 0.4384
eval_f1: 0.4733
eval_runtime: 651.4065
eval_samples_per_second: 62.9910
eval_steps_per_second: 3.9380
epoch: 1.0000


In [9]:
# Save final model
final_model_path = '/kaggle/working/models/task2_bert_final'
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)

print(f"✅ Model saved to: {final_model_path}")
print("\n💡 TIP: Click 'Save Version' to commit and save this model permanently!")
print("\n" + "="*60)
print("✅ TASK 2 - BERT COMPLETE!")
print("="*60)

✅ Model saved to: /kaggle/working/models/task2_bert_final

💡 TIP: Click 'Save Version' to commit and save this model permanently!

✅ TASK 2 - BERT COMPLETE!
